In [1]:
import pandas as pd


## Cargar, limpiar y organizar los datos

In [ ]:
artistas = pd.read_csv('../data/artists_es.csv')

artistas.iloc[1000:1005]

In [ ]:
artistas.shape

In [ ]:
artistas['generos'].iloc[1000:1005]

In [ ]:
type(artistas['generos'].iloc[1002])

La función $ast.literal\_eval()$ interpreta un str que tenga alguno de los tipos de datos básicos de Python y lo convierte en el tipo de dato interpretado.

In [6]:
import ast
artistas['generos'] = artistas['generos'].apply(lambda generos_str: ast.literal_eval(generos_str))

In [ ]:
type(artistas['generos'].iloc[1002])

---

In [ ]:
tracks = pd.read_csv('../data/tracks_es.csv')

tracks.head()

Corrección de tipo de variable en las columans `artistas` y `id_artistas`.

In [9]:
tracks['artistas'].iloc[50], tracks['id_artistas'].iloc[50]

("['Fernando Pessoa']", "['14jtPCOoNZwquk5wd9DxrY']")

In [10]:
type(tracks['artistas'].iloc[50]), type(tracks['id_artistas'].iloc[50])

(str, str)

### Avance 1

- Limpiando y ordenando los datos
- Agregando columnas apartir de los datos de otras columnas

In [11]:
tracks['artistas'] = tracks['artistas'].apply(lambda artistas_str: ast.literal_eval(artistas_str))
tracks['id_artistas'] = tracks['id_artistas'].apply(lambda artistas_str: ast.literal_eval(artistas_str))

### Agregando columna de año(year) de lanzamiento a partir de la columna fecha_lanzamiento 

In [ ]:
tracks['year'] = tracks['fecha_lanzamiento'].apply(lambda fecha_lanzamiento: int(fecha_lanzamiento[:4]))

# imprimos el nuevo dataframe con la nueva columna year
tracks.head(5)

### filtrando por columnas

In [ ]:
artistas.sample(5)


In [ ]:
tracks.sample(5)

In [ ]:
artistas['generos'].explode().value_counts().head(20)

In [ ]:
GENERO_ELEGIDO = 'harp'

artistas['genero en lista'] = artistas['generos'].apply(lambda generos: GENERO_ELEGIDO in generos)
artistas.head(20)

### filtrando los artistas con el genero escogido

In [ ]:
filtro = artistas['genero en lista'] == True

select_artistas = artistas.loc[filtro]
select_artistas.sample(10)

In [18]:
select_artistas.shape

(82, 6)

In [19]:
lista_artistas_ids = select_artistas['id'].tolist()
len(lista_artistas_ids)

82

In [20]:
# Agregando la nueva columnas a tracks con los id de los artistas validos
tracks['id_artistas_validos'] = tracks['id_artistas'].apply(lambda artista: any(artista in lista_artistas_ids for artista in artista))

tracks_validos = tracks.loc[tracks['id_artistas_validos']]
tracks_validos.shape

(46, 22)

In [ ]:
tracks_validos.head()

In [ ]:
# obteniendo el artista principal de cada track

tracks_validos['artista principal'] = tracks_validos['artistas'].apply(lambda artistas: artistas[0])
tracks_validos.head()

## Filtrando columnas de los tracks validos

- Filtrando solo las columnas de interes

### Avance 2

In [23]:
tracks_validos = tracks_validos.drop(columns=['id',   'explicita', 'artistas',  'id_artistas',  'fecha_lanzamiento', 'key', 'modo', 'valencia', 'tempo',  'compas', 'id_artistas_validos'])

- Visualizando el nuevo dataframe sin las columnas de no interes

In [ ]:
tracks_validos.head()

## Agrupaciones de registros

In [ ]:
# promedio 

tracks_validos[['nombre', 'popularidad', 'duracion_ms']].groupby('nombre').agg({
    'popularidad': 'mean',
    'duracion_ms': 'sum'
})

In [ ]:
# Tablas dinamicas
tracks_validos.pivot_table(index=['nombre', 'popularidad'],
                           values='duracion_ms',
                           aggfunc='mean')


## Avance 3

In [ ]:

# Seleccionando los artistas con mas popularidad

artistas_populares = tracks_validos[['artista principal', 'popularidad']].groupby('artista principal').max().sort_values(
    by='popularidad', ascending=False
).head().reset_index()

print('Top artistas populares')
artistas_populares

In [ ]:
# almacenando en una lista los nombres de los artistas populares
list_artistas_plot = artistas_populares['artista principal'].tolist()
list_artistas_plot

In [ ]:
tracks_plot = tracks_validos.loc[tracks_validos['artista principal'].isin(list_artistas_plot)]
tracks_plot = tracks_plot.loc[tracks_plot['year'] > 1970] #al tratarse de musica clasica se adapto el filtro del year
tracks_plot.shape

In [ ]:
# midiendo la popularidad promedio de los artistas seleccionados
tracks_plot[['artista principal', 'popularidad']].groupby('artista principal').mean()

## creando tabla dinamica con artista y year con el promedio de la popularidad

In [ ]:
tabla_popularidad = tracks_validos.pivot_table(index=['artista principal', 'year'],
                                               values='popularidad',
                                               aggfunc='mean')

tabla_popularidad

## Visualizacion de datos